<a href="https://colab.research.google.com/github/bongho/Project_ASAC_5th_Review/blob/hoon/%ED%85%8D%EC%8A%A4%ED%8A%B8_%EC%9A%94%EC%95%BD_yelp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 텍스트를 요약(Summary)하고 키워드를 추출해보기

In [2]:
!pip install datasets
!pip install transformers==4.17

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 23.8 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 70.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4


In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import Trainer, TrainingArguments
from sklearn.model_selection import train_test_split

import torch
import pandas as pd
import numpy as np

# 데이터 준비 및 전처리

In [ ]:
review = pd.read_csv('/content/drive/MyDrive/ASAC 데이터 분석 과정/기업연계_프로젝트/encoded_review_df_filtered.csv')
review.head(1)

df = review

In [8]:
### 가게 리뷰 groupby 통합

temp_df = df.drop(['review_id','user_id', 'date'], axis=1)
group_temp = temp_df.groupby('business_id').agg({
    'text': ' '.join,
    'stars': ['count','mean'],
    'useful': 'sum',
    'funny': 'sum',
    'cool': 'sum'
    }).reset_index()

# T5 모델 리뷰 요약(summarization)

- 통일된 접근법: T5는 텍스트를 텍스트로 변환하는 일관된 접근 방식을 사용하여 키워드 생성 등 다양한 작업을 수행
- 다양한 작업 수행 능력: 텍스트 요약, 번역, 질문 생성, 키워드 생성 등 여러 작업을 하나의 모델로 수행

In [ ]:
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration
from multiprocessing import Pool, set_start_method, cpu_count
from tqdm import tqdm
import numpy as np
import torch

# 'spawn' 시작 방법 설정
set_start_method('spawn', force=True)

# 1. 토크나이저와 모델 로드
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

In [ ]:
# 2. 텍스트 요약 함수 정의
def summarize_text(text):
    input_text = "summarize: " + text
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)
    outputs = model.generate(input_ids=input_ids, max_length=150, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary

## 결과 확인

In [ ]:
summarize_text(df['text'][0])

'we have tried it multiple times, because I want to like it! the food is good, but it takes a very long time to come out.'

,business_id,stars,useful,funny,cool,text
0,80739,3.0,0,0,0,"If you decide to eat here, just be aware it is..."
1,98537,4.0,1,0,1,Cute interior and owner (?) gave us tour of up...
2,94651,4.0,1,0,1,I was really between 3 and 4 stars for this on...
3,92221,5.0,0,0,0,First time there and it was excellent!!! It fe...
4,92778,5.0,0,0,0,"Great burgers,fries and salad! Burgers have a..."
...,...,...,...,...,...,...
2680156,93796,1.0,1,2,0,"The supper club is ridiculously expensive. So,..."
2680157,121621,4.0,1,0,0,Took a colleague here for dinner as we were tr...
2680158,67194,4.0,2,0,1,This place never fails the food is absolutely ...
2680159,102240,3.0,1,1,0,This is a good pizza option - they deliver thr...


In [9]:
group_temp.columns = ['business_id', 'text', 'count','stars','useful', 'funny', 'cool']

In [ ]:
group_temp.sort_values('count', ascending=False)

In [ ]:
summarize_text(group_temp['text'][0])

Token indices sequence length is longer than the specified maximum sequence length for this model (1774 > 512). Running this sequence through the model will result in indexing errors


"a new local seafood spot in New Port Richie is a byob. the restaurant is a byob. it's the only place to get raw clams."

In [ ]:
summarize_text(group_temp['text'][2])

"humpty's is firmly on our blacklist. the service was fine, the food was DESPICABLE. the mashed potatoes tasted like acid reflux."

## 가게 리뷰 합산 요약

In [ ]:
group_temp['text'][2]
## 모델의 metric
# ROGUE
# 원문과 요약의 공통 비율
# ROGUE 1 : 1-gram안에서의 공통 비율

'Server was ignorant, and did the bare minimum. My boyfriend and I had the Swiss chicken wraps, and the lettuce was soggy and old, and the tomatoes were extremely underripe. I brought the issue to the waitresses attention and nothing was done to fix the issue. Overall a not so great experience. The fries & gravy were tolerable, but not enough for us to return. I definitely do not recommend this establishment. My wife and I had some time to spare before going to the Royal Alex hospital so we decided on Humpty\'s just across the street at 9AM. My wife ordered the waffle and I had the Canadian skillet. Coffee was plentiful. Orders arrived after 12 minutes which was ok considering the place was only about 1/4 full. Waffle topped with overly sweet whipped cream and some strawberries. I had a full skillet but 3/4 were potatoes. Couldn\'t really tell how many eggs were in the skillet as there was a very unusual and off tasting yellow sauce. Needed a magnifying glass to find the bacon. I left 

In [5]:
df['text']

0          If you decide to eat here, just be aware it is...
1          Cute interior and owner (?) gave us tour of up...
2          I was really between 3 and 4 stars for this on...
3          First time there and it was excellent!!! It fe...
4          Great burgers,fries and salad!  Burgers have a...
                                 ...                        
2680156    The supper club is ridiculously expensive. So,...
2680157    Took a colleague here for dinner as we were tr...
2680158    This place never fails the food is absolutely ...
2680159    This is a good pizza option - they deliver thr...
2680160    Located in the 'Walking District' in Nashville...
Name: text, Length: 2680161, dtype: object

# T5 모델 Keyword 요약(keyword extract)

In [6]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model = T5ForConditionalGeneration.from_pretrained("Voicelab/vlt5-base-keywords")
tokenizer = T5Tokenizer.from_pretrained("Voicelab/vlt5-base-keywords")

task_prefix = "Keywords: "
# inputs = [
#     "Christina Katrakis, who spoke to the BBC from Vorokhta in western Ukraine, relays the account of one family, who say Russian soldiers shot at their vehicles while they were leaving their village near Chernobyl in northern Ukraine. She says the cars had white flags and signs saying they were carrying children.",
#     "Decays the learning rate of each parameter group by gamma every step_size epochs. Notice that such decay can happen simultaneously with other changes to the learning rate from outside this scheduler. When last_epoch=-1, sets initial lr as lr.",
#     "Hello, I'd like to order a pizza with salami topping.",
# ]

for sample in df.loc[:10,'text']:
    input_sequences = [task_prefix + sample]
    input_ids = tokenizer(
        input_sequences, return_tensors="pt", truncation=True
    ).input_ids
    output = model.generate(input_ids, no_repeat_ngram_size=3, num_beams=4)
    predicted = tokenizer.decode(output[0], skip_special_tokens=True)
    print(sample, "\n --->", predicted)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


If you decide to eat here, just be aware it is going to take about 2 hours from beginning to end. We have tried it multiple times, because I want to like it! I have been to it's other locations in NJ and never had a bad experience. 

The food is good, but it takes a very long time to come out. The waitstaff is very young, but usually pleasant. We have just had too many experiences where we spent way too long waiting. We usually opt for another diner or restaurant on the weekends, in order to be done quicker. 
 ---> NJ, food, quicker, waitstaff
Cute interior and owner (?) gave us tour of upcoming patio/rooftop area which will be great on beautiful days like today. Cheese curds were very good and very filling. Really like that sandwiches come w salad, esp after eating too many curds! Had the onion, gruyere, tomato sandwich. Wasn't too much cheese which I liked. Needed something else...pepper jelly maybe. Would like to see more menu options added such as salads w fun cheeses. Lots of beer

In [10]:
for sample in group_temp.loc[:10,'text']:
    input_sequences = [task_prefix + sample]
    input_ids = tokenizer(
        input_sequences, return_tensors="pt", truncation=True
    ).input_ids
    output = model.generate(input_ids, no_repeat_ngram_size=3, num_beams=4)
    predicted = tokenizer.decode(output[0], skip_special_tokens=True)
    print(sample, "\n --->", predicted)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Ate here for the 1st time on Saturday 08/07/2020. 
The food was ok, had the Lobster Boil . The Lobster was ok for a soft shelled Lobster. The Crabs were soggy and mushy. The Clams were gummy and snotty . The headed shrimp were large and tasty. All the items were precooked, obviously. The fish spread was decent , also had the Fried Shrimp and Shrimp scampi  which were decent. BYOB ... ... The atmosphere was nice except for the zillion Flies. I'll be back with a six pack Absolutely fantastic!!!!  I'll be back for sure. Fried lobster tail was delicious. Five star for sure. I was sadly disappointed in my experience. We ordered crabs and they were extremely small, watery and over half were bad(sick crabs), we ordered beans and rice which lacked flavor and literally had one bean in it that you can see from the picture. The coleslaw also lacked taste. We ask for grouper nuggets and a few were not cooked all the way. So given the food experience it was nothing like the reviews we read. Why I a

# 2. Bert로 핵심 문장 추출 한 뒤 여러 알고리즘으로 keyword 추출

- 양방향 문맥 이해: BERT는 양방향 문맥을 고려하여 텍스트를 이해하는 것에 효과적이다.
- 주로 텍스트 분류, 질문 답변, 문장 관계 예측에 사용됨.

[참고] https://iopscience.iop.org/article/10.1088/1742-6596/1992/4/042077/pdf

In [13]:
from transformers import BertModel, BertTokenizer
import torch
from sklearn.metrics.pairwise import cosine_similarity

# Bert 모델과 토크나이저 로드
model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def extract_key_sentences(text, num_sentences=3):
    sentences = text.split('.')
    embeddings = []

    for sentence in sentences:
        inputs = tokenizer(sentence, return_tensors='pt')
        outputs = model(**inputs)
        embeddings.append(outputs.last_hidden_state.mean(dim=1).squeeze().detach().numpy())

    similarities = cosine_similarity(embeddings)
    key_sentences = sorted(range(len(similarities)), key=lambda i: similarities[i].sum(), reverse=True)[:num_sentences]

    return [sentences[i] for i in key_sentences]

In [14]:
extract_key_sentences(group_temp['text'][0])

[' \n\nI ordered to go my order was ready in 10 minutes \n\nThe staff was very polite \n\nMy food was hot, fresh and tasty! The potion of coleslaw was huge an my lobster was a nice size',
 " Can't wait to return!! Great little spot and best of all BYOB !!!!\nLove it \n\nIs the food was fresh coat and they brought in the stone crabs that night on ice !  The boiled shrimp are great and the outside seating makes it feel very old Florida which is cool  , oysters Rockefeller was  pretty good",
 " The establishment makes you feel so comfortable and warm you don't want to leave! I had the Cuban Empanadas's and they were delicious  I ordered the baked beans also and they were some of the best I've ever had"]

In [15]:
extract_key_sentences(df['text'][0])

[' \n\nThe food is good, but it takes a very long time to come out',
 ' We usually opt for another diner or restaurant on the weekends, in order to be done quicker',
 'If you decide to eat here, just be aware it is going to take about 2 hours from beginning to end']

In [16]:
df['text'][0]

"If you decide to eat here, just be aware it is going to take about 2 hours from beginning to end. We have tried it multiple times, because I want to like it! I have been to it's other locations in NJ and never had a bad experience. \n\nThe food is good, but it takes a very long time to come out. The waitstaff is very young, but usually pleasant. We have just had too many experiences where we spent way too long waiting. We usually opt for another diner or restaurant on the weekends, in order to be done quicker."

### TF-IDF, TextRank, LDA 알고리즘 사용 후 keyword 결합하여 추출

In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import networkx as nx

# TF-IDF 키워드 추출
def tfidf_keywords(text, num_keywords=5):
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform([text])
    indices = X.toarray().argsort()[0, -num_keywords:]
    features = vectorizer.get_feature_names_out()
    return [features[i] for i in indices]

# TextRank 키워드 추출
def textrank_keywords(text, num_keywords=5):
    words = text.split()
    word_graph = nx.Graph()

    for i, word in enumerate(words):
        for j in range(i+1, min(i+5, len(words))):
            word_graph.add_edge(word, words[j])

    scores = nx.pagerank(word_graph)
    ranked_words = sorted(scores, key=scores.get, reverse=True)[:num_keywords]
    return ranked_words

# LDA 키워드 추출
def lda_keywords(text, num_keywords=5):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform([text])
    lda = LatentDirichletAllocation(n_components=1)
    lda.fit(X)
    indices = lda.components_[0].argsort()[-num_keywords:]
    features = vectorizer.get_feature_names_out()
    return [features[i] for i in indices]


import nltk
from nltk.corpus import stopwords

# 불용어 다운로드
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)



# 최종 키워드 결합
def extract_keywords(text, num_keywords=5):
    key_sentences = extract_key_sentences(text)
    combined_text = ' '.join(key_sentences)
    preprocessed_text = preprocess_text(combined_text)

    tfidf_kw = tfidf_keywords(combined_text, num_keywords)
    textrank_kw = textrank_keywords(combined_text, num_keywords)
    lda_kw = lda_keywords(combined_text, num_keywords)

    return list(set(tfidf_kw) | set(textrank_kw) | set(lda_kw))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

# 리뷰 데이터를 LDA를 사용하여 토픽 모델링
def topic_modeling(reviews, num_topics=5, num_keywords=5):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(reviews)

    lda = LatentDirichletAllocation(n_components=num_topics)
    lda.fit(X)

    topics = []
    for idx, topic in enumerate(lda.components_):
        keywords = [vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-num_keywords - 1:-1]]
        topics.append(keywords)

    return topics

#### 불용어 처리 전 예시

In [40]:
# # 예시 데이터
reviews = [
    "The food was great and the service was excellent.",
    "I love the ambiance but the food was just okay.",
    "The pasta was delicious and the wine selection was superb."
]

# 토픽별 키워드 추출
topics = topic_modeling(reviews, num_topics=3, num_keywords=5)
for idx, topic in enumerate(topics):
    print(f"Topic {idx+1}: {topic}")


extract_keywords(reviews[0])

Topic 1: ['the', 'was', 'wine', 'superb', 'selection']
Topic 2: ['food', 'and', 'the', 'was', 'ambiance']
Topic 3: ['was', 'the', 'food', 'and', 'great']


['food', 'great', 'was', 'and', 'the', 'service']

#### 불용어 처리 후

In [43]:
# 예시 데이터
reviews = [
    "The food was great and the service was excellent.",
    "I love the ambiance but the food was just okay.",
    "The pasta was delicious and the wine selection was superb."
]

preprocessed_reviews = [preprocess_text(review) for review in reviews]
topics_v2 = topic_modeling(preprocessed_reviews, num_topics=3, num_keywords=5)
for idx, topic in enumerate(topics_v2):
    print(f"Topic {idx+1}: {topic}")

extract_keywords(preprocessed_reviews[0])

Topic 1: ['food', 'love', 'okay', 'ambiance', 'excellent']
Topic 2: ['wine', 'superb', 'selection', 'pasta', 'delicious']
Topic 3: ['service', 'great', 'excellent', 'love', 'ambiance']


['food', 'great', 'excellent', 'service']

#### 실제 리뷰 데이터로 실험

In [29]:
preprocessed_reviews = [preprocess_text(review) for review in df.loc[:1,'text']]

topics_v2 = topic_modeling(preprocessed_reviews, num_topics=3, num_keywords=5)
for idx, topic in enumerate(topics_v2):
  print(f"Topic {idx+1}: {topic}")

Topic 1: ['like', 'curds', 'cheese', 'come', 'good']
Topic 2: ['waitstaff', 'want', 'decide', 'way', 'done']
Topic 3: ['waitstaff', 'want', 'decide', 'way', 'done']


In [30]:
df.at[1,'text']

"Cute interior and owner (?) gave us tour of upcoming patio/rooftop area which will be great on beautiful days like today. Cheese curds were very good and very filling. Really like that sandwiches come w salad, esp after eating too many curds! Had the onion, gruyere, tomato sandwich. Wasn't too much cheese which I liked. Needed something else...pepper jelly maybe. Would like to see more menu options added such as salads w fun cheeses. Lots of beer and wine as well as limited cocktails. Next time I will try one of the draft wines."